# 机器学习策略（1）


## 1. 为什么需要机器学习策略

改进机器学习模型，可以做的事情有很多，效果各不相同。如何去分析机器学习问题，让模型更快地达到理想的状态，这是机器学习策略的意义。

![Motivating example](img/Motivating example.png)

## 2. 正交化

正交化是一种系统设计的特性，它可以保证算法的一个指令或组件，不会对系统的其它组件产生副作用。这个特性使得验证算法的工具各自独立，从而减轻测试和开发周期。

在设计监督学习系统时，下面是4个假设，我们需要这些假设都成立，并且相互正交：

1. 成本函数在训练集上表现良好
    - 如果不好，使用更大的神经网络，或者使用更好的优化算法
2. 成本函数在开发集上表现良好
    - 如果不好，使用正则化，或者使用更大的训练集
3. 成本函数在测试集上表现良好
    - 如果不好，使用更大的开发集
4. 模型在现实世界里表现良好
    - 如果不好，开发集可能设置错误，或者所选的成本函数不是衡量所给问题好坏的一个很好指标

![Chain of assumptions in ML](img/Chain of assumptions in ML.png)

## 3. 设立目标

### 3.1 单一的数字度量

设立唯一的实数度量，来衡量模型对当前的问题的表现好坏。比如在非均衡分类问题中，有精度和召回率等多种度量指标，通常会将其合并为一个F1-score的度量，否则无法挑选模型。

### 3.2 优化度量和满意度量

评价一个模型，除去优化的度量指标之外，通常我们还会关心一些现实世界的指标（来自产品、运营方面的要求），而非模型理论层面的优化目标。比如模型训练所需要的时间，比如训练分类器时可能想考虑过去24小时内FN出现的次数（这个是无法通过模型优化的）。

这些现实世界的指标，很难和优化度量一起合并为一个单一的数字度量。比如优化度量是准确率，不能说设立一个成本，等于准确率-0.5×训练时间，这种做法是不显示的。

实际的做法，通常是将现实世界的指标，作为一个满意度量，设立目标范围。在通过各个满意度量满足的前提下，比较优化度量。这样对于模型，可以有N个度量值，其中一个是优化度量，剩下的是带有约束调节的满意度量。

### 3.3 训练集、开发集、测试集的分布

开发集和测试集应当反应未来实际线上的数据场景，并尽力去优化开发集和测试集上成本函数的数值。至少要保证开发集和测试集来自同一分布。切忌比如按照区域、按照人群划分开发集和测试集，请从全体数据中随机抽取。

训练集有时为了获得更大的训练样本，可以适当放宽同一分布的约束。在前深度学习时代，从理论上，训练集也应该来自同一分布。

### 3.4 开发集和测试集的大小

传统机器学习时代，数据量的切分通常是70%训练集30%测试集，或者60%训练集20%开发集20%测试集。在大数据时代，由于数据量很大，更常见的是98%训练集1%开发集1%测试集。

- 测试集的大小要可以保证总体表现良好
- 测试集的大小可以小于整个数据集的30%
- 开发集需要足够大，可以区分不同模型的好坏

### 3.5 变更开发集、测试集或评价度量的时机

1. 评价指标不能反应模型实际的好坏：
    - 比如采用错误率作为猫图片分类器的评价度量。算法A的分类错误率是3%，而算法B的分类错误率是5%。从数值上看，算法A的表现更好。但是，算法A有一个副作用，它可能会将某些黄色图片错误分类为猫，从而透出给用户，而算法B没有这个问题。从现实的角度，其实算法B的表现更好。评价的度量应该改变。
    - 可以对错误分类的图片，再进行归类。其中是黄色图片的，增加对应损失函数的权重。从而获得更好的评价度量。
    - 针对重新定义的评价度量，进行优化。（意味着成本函数改变了，梯度的计算也会不同）

2. 生产环境的数据，和开发集测试集都不同：
    - 以猫的图片分类器为例，可能开发集测试集用的图片是网上爬取的高清图片，而生产环境的数据由用户上传，分辨率不高。模型对于分辨率不高的数据，错误率大大上升。
    - 在开发集、测试集上表现良好的度量指标，不能反映线上的表现情况。需要更改开发集、测试集数据，或者改变度量指标。

## 4. 对比人类的水准

### 4.1 为什么要参考人类水准

如今机器学习算法已经可以在很多问题上达到人类的水平。同时，很多问题上，由于人所能达到的水准已经接近完美了，所以很多情况下机器学习算法都在力图达到人类可以达到的水平。

![Why human-level performance](img/Why human-level performance.png)

就具体问题而言，一旦机器学习发展到超越人类水平之后，进展就迅速减缓了。一方面是因为人类水准常常已经非常接近**贝叶斯最优误差 Bayes optimal error**，尤其是针对图像、声音、语言这些问题。贝叶斯最优误差定义为可能达到的最优误差，换言之，它是从X到Y的最优函数映射，这个映射的准确率无法被超越。

另一方面的原因是，在超越人类水平后，很多之前可以用来提升算法的工具都不再能用上。比如：
    - 获取人类打标好的数据
    - 进行人工错误分析，得知为什么人们会犯错
    - 更好地分析偏差和方差